In [ ]:
import pandas as pd
import os
import re
from Bio import SeqIO
import subprocess 
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

%matplotlib inline


In [ ]:
Dir='/nasdata/Vincent/Strep/nanopore/'
os.chdir(Dir)
nanoporeData='/nasdata/Vincent/Strep/nanopore/PAC38/fastq_pass/'

mergedSeqFile=Dir+'PAC38.fastq'
cmd='cat '+nanoporeData+'*.fastq > '+mergedSeqFile
print cmd
subprocess.call(cmd, shell= True) 

In [ ]:
vectorFa='/nasdata/Strep/nanopore/pESAC-Apramycin_16kbCore.fasta'
fastqOut='PAC38_combined_all_clipped_vector.fastq'

#find vector in sequences 
cmd="minimap2 -a "+vectorFa+" "+mergedSeqFile+" | samtools sort -@ 60 -o ALL.sorted.bam -"
subprocess.call(cmd, shell= True) 
subprocess.call(["samtools view -@ 60 -b -F 4 ALL.sorted.bam > vector_mapped.bam"], shell= True)
subprocess.call(["samtools view -b -f 4 ALL.sorted.bam > unmapped.bam"], shell= True)
#contigs without vector are kept as is
subprocess.call(["samtools bam2fq unmapped.bam > unmapped.fastq"], shell= True)

#contigs with vector have their non-aligned (clipped) parts extracted
subprocess.call(["java -jar /home/vlibis/jvarkit/dist/samextractclip.jar --minsize 1000 -o /nasdata/Vincent/Strep/nanopore/soft_clipped.fq /nasdata/Vincent/Strep/nanopore/vector_mapped.bam"], shell= True)

#merge contigs without vector and non-vector clipped parts from the other contigs
cmd ="cat soft_clipped.fq unmapped.fastq > "+fastqOut
print cmd
subprocess.call(cmd, shell= True)

cmd='seqkit seq '+fastqOut+' -m 1000 > sized_'+fastqOut
print cmd
subprocess.call(cmd, shell= True)

cmd='rename.sh in=sized_'+fastqOut+' out=renamed_sized_'+fastqOut+' prefix=nano'
print cmd
subprocess.call(cmd, shell= True)
print 'renamed_sized_'+fastqOut

In [ ]:
#run FLYE assembler
cmd="flye --nano-raw /nasdata/Vincent/Strep/nanopore/renamed_sized_PAC38_combined_all_clipped_vector.fq --out-dir FLYE_PAC38 --threads 60"
subprocess.call(cmd, shell= True) 